# 一、从问题中提取相关实体和关系

In [2]:
import os
import ahocorasick
from tqdm import tqdm

导入相关的包 这里导入的ahocorasick是AC自动机，可以支持多模式下的匹配任务
详细的AC自动机算法机制：https://oi-wiki.org/string/ac-automaton/#SnippetTab

In [3]:
os.chdir('QAMedicalKG')
cur_dir = os.getcwd()#这一行代码的作用是获取当前工作目录，并将其存储在变量

改变操作路径，按照我的项目结构进行配置，这句就不会报错

In [4]:
#　载入所有的特征词文件
disease_path = os.path.join(cur_dir, 'dict/disease.txt') # 没有啥意义的连接路径操作
department_path = os.path.join(cur_dir, 'dict/department.txt')
check_path = os.path.join(cur_dir, 'dict/check.txt')
drug_path = os.path.join(cur_dir, 'dict/drug.txt')
food_path = os.path.join(cur_dir, 'dict/food.txt')
producer_path = os.path.join(cur_dir, 'dict/producer.txt')
symptom_path = os.path.join(cur_dir, 'dict/symptom.txt')
deny_path = os.path.join(cur_dir, 'dict/deny.txt')

按照路径进特征词文件的检索以及载入，其实这里比较麻烦，如果不改变路径，可以不用连接两个地址，直接进行文件的读取和载入

In [4]:
# 加载特征词 使用列表生成式和 strip() 方法，去除每行的前后空白字符，并过滤掉空行
disease_wds = [i.strip() for i in open(disease_path,encoding="utf-8") if i.strip()]
department_wds = [i.strip() for i in open(department_path,encoding="utf-8") if i.strip()]
check_wds = [i.strip() for i in open(check_path,encoding="utf-8") if i.strip()]
drug_wds = [i.strip() for i in open(drug_path,encoding="utf-8") if i.strip()]
food_wds = [i.strip() for i in open(food_path,encoding="utf-8") if i.strip()]
producer_wds = [i.strip() for i in open(producer_path,encoding="utf-8") if i.strip()]
symptom_wds = [i.strip() for i in open(symptom_path,encoding="utf-8") if i.strip()]
# 将所有特征词合并成一个集合，集合会自动去重
region_words = set(disease_wds + department_wds + check_wds + drug_wds + food_wds + producer_wds + symptom_wds)
deny_words = [i.strip() for i in open(deny_path,encoding="utf-8") if i.strip()]

这里是对特征词进行简单的数据处理使用列表生成式和 strip() 方法，去除每行的前后空白字符，并过滤掉空行

In [5]:
#为每个特征词建立一个字典，记录每个词的类型。通过遍历所有特征词，将词所属的类型添加到对应的字典中
wdtype_dict = dict()
for word in tqdm(region_words):
    wdtype_dict[word] = []
    if word in disease_wds:
        wdtype_dict[word].append('disease')
    if word in department_wds:
        wdtype_dict[word].append('department')
    if word in check_wds:
        wdtype_dict[word].append('check')
    if word in drug_wds:
        wdtype_dict[word].append('drug')
    if word in food_wds:
        wdtype_dict[word].append('food')
    if word in symptom_wds:
        wdtype_dict[word].append('symptom')
    if word in producer_wds:
        wdtype_dict[word].append('producer')

100%|██████████████████████████████████████████████████████████████████████████| 43430/43430 [00:27<00:00, 1574.58it/s]


手动生成为每个特征词建立一个字典，记录每个词的类型。通过遍历所有特征词，将词所属的类型添加到对应的字典中，可以使用print(wdtype_dict)查看生成情况
这里生成的字典是后面进行分类的依据

# 当知识图谱规模过大时，使用AC自动机算法，从问题中提取知识图谱中的实体，而不是用原生python的字符串方法

In [6]:
#构建 Aho-Corasick 自动机。Aho-Corasick 自动机是一种多模式字符串匹配算法，能够高效地在文本中找到多个模式
def build_actree(wordlist):
    actree = ahocorasick.Automaton() #建立空的自动机
    for index, word in enumerate(wordlist):
        actree.add_word(word, (index, word)) #将每个单词添加到自动机中
    actree.make_automaton() #对所有添加的单词进行处理，构建出自动机的状态转移图
    return actree #回构建好的 Aho-Corasick 自动机实例

这里使用AC自动机构建一个多模式下的字符串匹配的函数，该函数生成一个自动机actree
在后面使用该函数对项目的特征词库进行处理
处理后的类型是一个二维元组（特征词（索引，特征词））

In [11]:
# 针对所有的实体构造AC自动机
region_tree = build_actree(list(region_words))

使用函数对特征词库进行处理

## 使用自动机：1.提取问题相关实体及其类别

In [12]:
question = '肺气肿和百日咳要做血常规吗'
question_entity = []
#使用自动机对question进行迭代匹配，返回一个迭代器
for each in region_tree.iter(question): 
    print(each)
    entity = each[1][1]   #each是一个元组(word, (index, word))
    question_entity.append(entity)
print(question_entity) #显示问题中出现的所有实体

(2, (41296, '肺气肿'))
(6, (40914, '百日咳'))
(11, (37371, '血常规'))
['肺气肿', '百日咳', '血常规']


这里输入了一个问题的示例 '肺气肿和百日咳要做血常规吗'
先构造一个空实体列表，将自动机进行迭代化，遍历生成的迭代器，对每一个出现的实体名进行收集，添加在实体列表中。
获得一个实体名构成的实体列表

In [13]:
#提取类别，匹配实体和类别
question_entity_dict = {each:wdtype_dict[each] for each in question_entity}
question_entity_dict

{'肺气肿': ['disease'], '百日咳': ['disease'], '血常规': ['check']}

根据上面提取的实体列表和之前提取出的实体-关系字典，进行匹配，可以获得该问题的一个字典question_entity_dict
这个字典反映了实体及其类别

In [14]:
#question_entity_dict被添加到输出字典的args中
output = {}
output['args'] = question_entity_dict

将问题中提出的实体名和实体类型进行存储在output中，以便下一阶段使用

## 使用自动机：2.提取问题相关的待查询关系

In [16]:
#问题涉及的实体类别
types = []
for each in question_entity_dict.values():
    types.extend(each)

['disease', 'disease', 'check']

再次利用question_entity_dict，问题的实体-类型关系字典 遍历，将问题中的特征词的类型单独提取出来，添加到types列表中。

In [17]:
#加载不同提问意图的疑问词列表
symptom_qwds = ['症状', '表征', '现象', '症候', '表现']
cause_qwds = ['原因','成因', '为什么', '怎么会', '怎样才', '咋样才', '怎样会', '如何会', '为啥', '为何', '如何才会', '怎么才会', '会导致', '会造成']
acompany_qwds = ['并发症', '并发', '一起发生', '一并发生', '一起出现', '一并出现', '一同发生', '一同出现', '伴随发生', '伴随', '共现']
food_qwds = ['饮食', '饮用', '吃', '食', '伙食', '膳食', '喝', '菜' ,'忌口', '补品', '保健品', '食谱', '菜谱', '食用', '食物','补品']
drug_qwds = ['药', '药品', '用药', '胶囊', '口服液', '炎片']
prevent_qwds = ['预防', '防范', '抵制', '抵御', '防止','躲避','逃避','避开','免得','逃开','避开','避掉','躲开','躲掉','绕开',
                     '怎样才能不', '怎么才能不', '咋样才能不','咋才能不', '如何才能不',
                     '怎样才不', '怎么才不', '咋样才不','咋才不', '如何才不',
                     '怎样才可以不', '怎么才可以不', '咋样才可以不', '咋才可以不', '如何可以不',
                     '怎样才可不', '怎么才可不', '咋样才可不', '咋才可不', '如何可不']
lasttime_qwds = ['周期', '多久', '多长时间', '多少时间', '几天', '几年', '多少天', '多少小时', '几个小时', '多少年']
cureway_qwds = ['怎么治疗', '如何医治', '怎么医治', '怎么治', '怎么医', '如何治', '医治方式', '疗法', '咋治', '怎么办', '咋办', '咋治']
cureprob_qwds = ['多大概率能治好', '多大几率能治好', '治好希望大么', '几率', '几成', '比例', '可能性', '能治', '可治', '可以治', '可以医']
easyget_qwds = ['易感人群', '容易感染', '易发人群', '什么人', '哪些人', '感染', '染上', '得上']
check_qwds = ['检查', '检查项目', '查出', '检查', '测出', '试出']
belong_qwds = ['属于什么科', '属于', '什么科', '科室']
cure_qwds = ['治疗什么', '治啥', '治疗啥', '医治啥', '治愈啥', '主治啥', '主治什么', '有什么用', '有何用', '用处', '用途', '有什么好处', '有什么益处', '有何益处', '用来', '用来做啥', '用来作甚', '需要', '要']

由于问题是一句疑问句，如何确定该回答什么还是需要根据疑问词来确定，所以加载一个提前准备好的疑问词列表库，一共有13个列表，13个列表对应了13种不同类型的问题

In [18]:
#检查提问语句中的疑问类词语
def check_words(words, question):
    # 疑问词是否出现在提问中
    for word in words:
        if word in question:
            # print(word + ' 出现在提问中')
            return True
    return False

建立一个检查函数，当疑问词在提问中，返回 真 否则返回 假

In [19]:
# 使用检查疑问词语句，提取关系
question_types = []


# #当问题中出现对应的症状疑问词且问题中有疾病实体
if check_words(symptom_qwds, question) and ('disease' in types):
    question_type = 'disease_symptom'
    question_types.append(question_type)
    
# 症状-疾病
if check_words(symptom_qwds, question) and ('symptom' in types):
    question_type = 'symptom_disease'
    question_types.append(question_type)
    
# 疾病-病因
if check_words(cause_qwds, question) and ('disease' in types):
    question_type = 'disease_cause'
    question_types.append(question_type)

# 疾病-并发症
if check_words(acompany_qwds, question) and ('disease' in types):
    question_type = 'disease_acompany'
    question_types.append(question_type)

# 疾病-可以吃/不可以吃
if check_words(food_qwds, question) and 'disease' in types:
    deny_status = check_words(deny_words, question)
    if deny_status:
        question_type = 'disease_not_food'
    else:
        question_type = 'disease_do_food'
    question_types.append(question_type)
    
# 可以吃/不可以吃-疾病
if check_words(food_qwds+cure_qwds, question) and 'food' in types:
    deny_status = check_words(deny_words, question)
    if deny_status:
        question_type = 'food_not_disease'
    else:
        question_type = 'food_do_disease'
    question_types.append(question_type)
    
# 疾病-药物
if check_words(drug_qwds, question) and 'disease' in types:
    question_type = 'disease_drug'
    question_types.append(question_type)
    
# 药物-疾病
if check_words(cure_qwds, question) and 'drug' in types:
    question_type = 'drug_disease'
    question_types.append(question_type)
    
# 疾病-检查
if check_words(check_qwds, question) and 'disease' in types:
    question_type = 'disease_check'
    question_types.append(question_type)
    
# 检查-疾病
if check_words(check_qwds+cure_qwds, question) and 'check' in types:
    question_type = 'check_disease'
    question_types.append(question_type)
    
#　疾病-预防措施
if check_words(prevent_qwds, question) and 'disease' in types:
    question_type = 'disease_prevent'
    question_types.append(question_type)
    
# 疾病-疗程
if check_words(lasttime_qwds, question) and 'disease' in types:
    question_type = 'disease_lasttime'
    question_types.append(question_type)

# 疾病-疗法
if check_words(cureway_qwds, question) and 'disease' in types:
    question_type = 'disease_cureway'
    question_types.append(question_type)

# 疾病-治愈率
if check_words(cureprob_qwds, question) and 'disease' in types:
    question_type = 'disease_cureprob'
    question_types.append(question_type)

# 疾病-易感人群
if check_words(easyget_qwds, question) and 'disease' in types :
    question_type = 'disease_easyget'
    question_types.append(question_type)

检查问题语句，逐个关系进行，一共15个检查，当问题的疑问词出现在一类疑问列表种且列表的类型对应到特征词的类别时，认为该关系可以被提取，被提取出来的关系添加在question_types中。

In [20]:
#关系
question_types

['check_disease']

这里检查出，问题 '肺气肿和百日咳要做血常规吗'属于的是check_disease 检查和疾病关系

In [21]:
# 若没有查到相关的外部查询信息，那么则将该疾病的描述信息返回
if question_types == [] and 'disease' in types:
    question_types = ['disease_desc']

# 若没有查到相关的外部查询信息，那么则将该疾病的描述信息返回
if question_types == [] and 'symptom' in types:
    question_types = ['symptom_disease']

# 将多个分类结果进行合并处理，组装成一个字典
output['question_types'] = question_types

In [22]:
output

{'args': {'肺气肿': ['disease'], '百日咳': ['disease'], '血常规': ['check']},
 'question_types': ['check_disease']}

到这里为止，整个问题已经被分类为，output中包含了问题设计到的关键词、关键词所属的类别以及问题对应的关系。

# 二、生成Cypher查询语句

In [25]:
output_pre=output
args = output_pre['args']

没啥特别的意义，就是将output的值分解出来

In [27]:
entity_dict = {}
for arg, types in args.items():
    for type in types:
        if type not in entity_dict:
            entity_dict[type] = [arg]
        else:
            entity_dict[type].append(arg)
entity_dict

{'disease': ['肺气肿', '百日咳'], 'check': ['血常规']}

生成一个字典entity_dict 为后续的问题解析为查询语句中的实体节点的匹配做准备

In [28]:
question_types = output_pre['question_types']

生成一个列表question_types 为后续的问题解析为查询语句中的关系的匹配做准备

In [29]:
question_type

'check_disease'

该问题只涉及到一个关系'check_disease'

In [30]:
sqls = []
#构建一个问题-cypher查询转化函数
#根据不同的问题类型，对应出不同的查询语句
def sql_transfer(question_type, entities):
    '''
    不同的提问意图，对应不同关系的cypher查询语句
    '''
    
    if not entities:
        return []

    # 查询语句
    sql = []
    # 查询疾病的病因
    if question_type == 'disease_cause':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cause".format(i) for i in entities]

    # 查询疾病的预防措施
    elif question_type == 'disease_prevent':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.prevent".format(i) for i in entities]

    # 查询疾病的疗程
    elif question_type == 'disease_lasttime':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cure_lasttime".format(i) for i in entities]

    # 查询疾病的治愈率
    elif question_type == 'disease_cureprob':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cured_prob".format(i) for i in entities]

    # 查询疾病的疗法
    elif question_type == 'disease_cureway':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cure_way".format(i) for i in entities]

    # 查询疾病的易感人群
    elif question_type == 'disease_easyget':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.easy_get".format(i) for i in entities]

    # 查询疾病的描述
    elif question_type == 'disease_desc':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.desc".format(i) for i in entities]

    # 查询疾病的症状
    elif question_type == 'disease_symptom':
        sql = ["MATCH (m:Disease)-[r:has_symptom]->(n:Symptom) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    # 查询症状对应的疾病
    elif question_type == 'symptom_disease':
        sql = ["MATCH (m:Disease)-[r:has_symptom]->(n:Symptom) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    # 查询疾病的并发症
    elif question_type == 'disease_acompany':
        sql1 = ["MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql2 = ["MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql = sql1 + sql2
    # 查询疾病的忌口
    elif question_type == 'disease_not_food':
        sql = ["MATCH (m:Disease)-[r:no_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    # 查询疾病建议吃的东西
    elif question_type == 'disease_do_food':
        sql1 = ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql2 = ["MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql = sql1 + sql2

    # 已知忌口查疾病
    elif question_type == 'food_not_disease':
        sql = ["MATCH (m:Disease)-[r:no_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    # 已知推荐查疾病
    elif question_type == 'food_do_disease':
        sql1 = ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql2 = ["MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql = sql1 + sql2

    # 查询疾病常用药品－药品别名记得扩充
    elif question_type == 'disease_drug':
        sql1 = ["MATCH (m:Disease)-[r:common_drug]->(n:Drug) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql2 = ["MATCH (m:Disease)-[r:recommand_drug]->(n:Drug) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql = sql1 + sql2

    # 已知药品查询能够治疗的疾病
    elif question_type == 'drug_disease':
        sql1 = ["MATCH (m:Disease)-[r:common_drug]->(n:Drug) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql2 = ["MATCH (m:Disease)-[r:recommand_drug]->(n:Drug) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql = sql1 + sql2
    # 查询疾病应该做的检查
    elif question_type == 'disease_check':
        sql = ["MATCH (m:Disease)-[r:need_check]->(n:Check) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    # 已知检查查询疾病
    elif question_type == 'check_disease':
        sql = ["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    return sql

构建一个问题-cypher查询转化函数
根据不同的问题类型，对应出不同的查询语句
question_type是之前在问题分类时候确定的问题的类型，这里用一个列表sql来存储查询语句。
最根本的查询语句是 
["MATCH (m:xxx)-[r:need_check]->(n:xxx) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
采用字符串格式化format的方法将提取出的实体和关系在查询语句中进行表现。

In [34]:
#使用函数生成查询语句
#一个字典 问题类型、语句
sqls = []
for question_type in question_types:
    sql_ = {}
    sql_['question_type'] = question_type
    sql_['sql'] = sql_transfer(question_type, entity_dict.get(question_type.split('_')[0]))
    sqls.append(sql_)
sqls

[{'question_type': 'check_disease',
  'sql': ["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '血常规' return m.name, r.name, n.name"]}]

执行该转化函数，将所有的查询语句都放在sqls中
本问题只有一个查询语句

# 三. 查询图数据库，生成回答

In [35]:
#连接知识图扑
from py2neo import Graph
g = Graph("bolt://localhost:xxxx", auth=("用户名", "密码"))

连接你自己的知识图扑，需要替换"bolt://localhost:xxxx"、"用户名"和 "密码"，一般的用户名应该都是neo4j

In [36]:
num_limit = 20
final_answers = []

num_limit = 20 是一个用于限制回答中元素数量的参数。在代码中，它用于控制最终回答中的列表元素数量，确保回答的可读性和简洁性。

In [37]:
#构建一个回复匹配函数
#根据问题类型自动进行回复的匹配
def answer_prettify(question_type, answers):
    final_answer = []
    if not answers:
        return ''
    if question_type == 'disease_symptom':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}的症状包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'symptom_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '症状{0}可能染上的疾病有：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_cause':
        desc = [i['m.cause'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}可能的成因有：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_prevent':
        desc = [i['m.prevent'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}的预防措施包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_lasttime':
        desc = [i['m.cure_lasttime'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}治疗可能持续的周期为：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_cureway':
        desc = [';'.join(i['m.cure_way']) for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}可以尝试如下治疗：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_cureprob':
        desc = [i['m.cured_prob'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}治愈的概率为（仅供参考）：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_easyget':
        desc = [i['m.easy_get'] for i in answers]
        subject = answers[0]['m.name']

        final_answer = '{0}的易感人群包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_desc':
        desc = [i['m.desc'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0},熟悉一下：{1}'.format(subject,  '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_acompany':
        desc1 = [i['n.name'] for i in answers]
        desc2 = [i['m.name'] for i in answers]
        subject = answers[0]['m.name']
        desc = [i for i in desc1 + desc2 if i != subject]
        final_answer = '{0}的症状包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_not_food':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}忌食的食物包括有：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_do_food':
        do_desc = [i['n.name'] for i in answers if i['r.name'] == '宜吃']
        recommand_desc = [i['n.name'] for i in answers if i['r.name'] == '推荐食谱']
        subject = answers[0]['m.name']
        final_answer = '{0}宜食的食物包括有：{1}\n推荐食谱包括有：{2}'.format(subject, ';'.join(list(set(do_desc))[:num_limit]), ';'.join(list(set(recommand_desc))[:self.num_limit]))

    elif question_type == 'food_not_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '患有{0}的人最好不要吃{1}'.format('；'.join(list(set(desc))[:num_limit]), subject)

    elif question_type == 'food_do_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '患有{0}的人建议多试试{1}'.format('；'.join(list(set(desc))[:num_limit]), subject)

    elif question_type == 'disease_drug':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}通常的使用的药品包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'drug_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '{0}主治的疾病有{1},可以试试'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_check':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}通常可以通过以下方式检查出来：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'check_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '通常可以通过{0}检查出来的疾病有{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    return final_answer

建立一个函数：
根据问题的类型自动匹配出答案的格式，由于直接查询出的节点直接用作回答可读性很差劲，所以需要进行一些处理，增强可读性，使其像一个答案。

In [38]:
#遍历列表，每个 sql_ 包含一个问题类型和对应的 Cypher 查询语句列表
for sql_ in sqls:
    question_type = sql_['question_type']
    queries = sql_['sql'] #保存当前问题类型对应的所有 Cypher 查询语句
    answers = []
    
    
    for query in queries: # 运行每一条cypher查询语句
        ress = g.run(query).data() #在知识图谱中进行查询
        answers += ress
    final_answer = answer_prettify(question_type, answers) #进行格式化，生成最终答案
    if final_answer:
        final_answers.append(final_answer)

将sqls中的每个查询语句都执行一遍，将查询结果进行格式化
最终的答案存在final_answer中

In [40]:
#查询结果 最初级的结果
answers

[{'m.name': '肺炎杆菌肺炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '肺炎球菌肺炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '放射性肺炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '二硫化碳中毒', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '大楼病综合征', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '大叶性肺炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '苯中毒', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '百日咳', 'r.name': '诊断检查', 'n.name': '血常规'}]

In [42]:
#返回答案 格式化后的答案
final_answer

'通常可以通过血常规检查出来的疾病有放射性肺炎；百日咳；二硫化碳中毒；肺炎球菌肺炎；肺炎杆菌肺炎；苯中毒；大叶性肺炎；大楼病综合征'